In [ ]:
import random
import re
import pandas as pd
from tqdm import tqdm
import numpy as np
import lightgbm as lgb
from pathlib import Path
from typing import Callable
from h3 import h3

In [ ]:
from utils import load_preprocessed_counters, load_supersegments, create_nodes_with_counters, \
    append_static_pred_relative_to_df, create_multi_traffic_state, split_train_valid, merge_pcas

# Fix paths and other global variables 

In [ ]:
data_dir = Path("/Users/andrei/Desktop/data4cast/data/")
# data_dir = Path("/Users/martin/PycharmProjects/traffic4cast/data/")

city_name = "madrid"
model_name = "extended_sota_previous"

CACHED = True
FULL_TRAIN = True

NEIGHBORS_FOR_WEIGHTING = 10
H3_RES = 6

# In submission, only got to set USE_SPEED_FEATURES=True for Melbourne.
#  We used USE_SPEED_FEATURES=False for Madrid, London.
USE_SPEED_FEATURES = True

# In submission, only got to set USE_ADVANCED_QUANTILES=True for London and Melbourne.
#  We used USE_ADVANCED_QUANTILES=False for Madrid.
USE_ADVANCED_QUANTILES = True

SAVE_MODEL_CHECKPOINTS = False

# Default count of iters to use when FULL_TRAIN is False
NO_LGB_ITERS = 800

num_iters = {
    "london": 5700,
    "madrid": 4900,
    "melbourne": 3200
}

num_leaves = {
    "london": 400,
    "madrid": 350,
    "melbourne": 350
}

# valid_weeks_hardcoded represents the weeks to use for validation when FULL_TRAIN is False
valid_weeks_hardcoded = {
    # From [23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53]
    "melbourne": [25, 33, 41, 49],
    # From [22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52]
    "madrid": [24, 32, 40, 48],
    # From [27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51,  1,  3,  5]
    "london": [29, 37, 45, 1]
}

# Fix static assets

In [ ]:
nodes = pd.read_parquet(data_dir / f"road_graph/{city_name}/road_graph_nodes.parquet")
node_coordinates = nodes.groupby("node_id")[["x", "y"]].first().to_dict(orient="index")
node_to_lat_lng = nodes.set_index("node_id")[["x", "y"]].T.to_dict()

In [ ]:
supersegments, supersegment_to_id, id_to_supersegment = load_supersegments(city_name, node_coordinates)

In [ ]:
nodes_with_counters = create_nodes_with_counters(city_name, blacklist=True)
nodes_with_counters

# Data creation

<h3> Create nearest counter features </h3>

In [ ]:
# Find nearest counter for edges which are not immediately at counter
from sklearn.neighbors import KDTree, BallTree
tree = KDTree(nodes_with_counters[["x", "y"]], metric="euclidean")
dist, ind = tree.query(supersegments[["x", "y"]], k=NEIGHBORS_FOR_WEIGHTING)

In [ ]:
supersegments["nearest_counter_id"] = ind[:,0]
# supersegments["nearest_counter_id"] = supersegments["nearest_counter_id"].astype("category")
supersegments["counter_distance_euclidean"] = dist[:,0]
supersegments["counter_distance_euclidean_mean_top5"] = dist[:,:5].mean(axis=1)
supersegments["counter_distance_euclidean_mean_all"] = dist.mean(axis=1)
supersegments

<h3> Utils for loading the train/test data and preparing them for downstream</h3>

In [ ]:
def load_data(city, mode="train"):
    counts = load_preprocessed_counters(city_name, mode)
    
    label_frames = []

    if mode == "train":
        files = sorted((data_dir / 'train' / city / 'labels').glob('eta_labels_*.parquet'))
        for f in files:
            label_frames.append(pd.read_parquet(f))
        labels = pd.concat(label_frames)
        print(labels.shape)
    else:
        labels = None
            
    return counts, labels

def normalize_data(mode="train"):
    if mode == "test":
        raise NotImplementedError
    
    counts, labels = load_data(city_name, mode=mode)
    labels["supersegment_id"] = [supersegment_to_id[s] for s in labels["identifier"]]
    del labels["identifier"]
    # Get supersegment features like nearest_counter_id
    labels = labels.merge(supersegments, on="supersegment_id")

    # Get counter_id of nodes
    counts = counts.merge(nodes_with_counters[["node_id", "counter_id"]], on="node_id")
    print(counts.shape)
    
    # Merge labels to nearest counters
    labels = labels.merge(counts, left_on=["day", "t", "nearest_counter_id"], right_on=["day", "t", "counter_id"])
    print(labels.shape)
    # Note that some time windows don't have counter data!
    # TODO left join, ignore nans?
    return labels

<h3> Util for H3 feature creation </h3>

In [ ]:
def create_h3_variable(data, mode="train"):
    counts = load_preprocessed_counters(city_name, mode)
    # counts["volumes_median"] = [np.median(v) for v in tqdm(counts["volumes_1h"])]
    counts["h3"] = [h3.geo_to_h3(node_to_lat_lng[x]["y"], node_to_lat_lng[x]["x"], H3_RES) for x in tqdm(counts["node_id"])]
    
    if mode == "train":
        counts_h3_volumes = counts.groupby(["day", "t", "h3"])[["volumes_mdn"]].sum()
    else:
        counts_h3_volumes = counts.groupby(["test_idx", "h3"])[["volumes_mdn"]].sum()
        
    time_hex_to_vol_dict = counts_h3_volumes.T.to_dict()
    data["h3"] = [h3.geo_to_h3(yy, xx, H3_RES) for xx, yy in tqdm(zip(data["x"], data["y"]))]
    if mode == "train":
        data["h3_vol"] = [
        time_hex_to_vol_dict.get(
            (x, y, z), 
            {"volumes_mdn": np.nan}
        )["volumes_mdn"] 
        for x, y, z in tqdm(zip(data["day"], data["t"], data["h3"]))]
    else:
        data["h3_vol"] = [
        time_hex_to_vol_dict.get(
            (x, z), 
            {"volumes_mdn": np.nan}
        )["volumes_mdn"] 
        for x, z in tqdm(zip(data["test_idx"], data["h3"]))]
    data["h3_vol"] = data["h3_vol"].fillna(data["h3_vol"].median())
    return data

<h3> Loading prepared training data if cached, loading raw data and preparing it otherwise </h3>

In [ ]:
if CACHED:
    try:
        data = pd.read_parquet(data_dir / "traffic" / city_name / "data_all_h3.parquet")
        print("Loaded data")
    except FileNotFoundError:
        data = normalize_data()
        data = create_h3_variable(data)
        data.to_parquet(data_dir / "traffic" / city_name / "data_all_h3.parquet")
else:
    data = normalize_data()
    data = create_h3_variable(data)
    data.to_parquet(data_dir / "traffic" / city_name / "data_all_h3.parquet")
    
data.count()

<h3> Appending PCA features </h3>

In [ ]:
data = merge_pcas(city_name, data)

### Splitting training data (if FULL_TRAIN=False)

In [ ]:
if FULL_TRAIN:
    train = data
else:
    train, valid = split_train_valid(city_name, data)
    del data

### Create ETA target encodings based on multiple granularities of conditional traffic quantiles

In [ ]:
traffic_quantiles_list = []

base_quantiles = [10, 20, 30, 40, 50, 100]
advanced_quantiles = [2, 4, 8, 10, 20, 30, 40, 50, 60, 80]

if USE_ADVANCED_QUANTILES:
    MULTI_LEVEL_QUANTILES = base_quantiles
else:
    MULTI_LEVEL_QUANTILES = advanced_quantiles

MULTI_LEVEL_QUANTILES = [2, 4, 8, 10, 20, 30, 40, 50, 60, 80]


for num_quantiles in tqdm(MULTI_LEVEL_QUANTILES):
    traffic_quantiles = train.groupby(["day", "t"])["volumes_mdn"].median().quantile(q=[i/num_quantiles for i in range(num_quantiles)])
    traffic_quantiles_list.append(traffic_quantiles)

In [ ]:
train, quantile_feats = create_multi_traffic_state(train, traffic_quantiles_list, "volumes_mdn", "train")
if not FULL_TRAIN:
    valid, _ = create_multi_traffic_state(valid, traffic_quantiles_list, "volumes_mdn", "train")

In [ ]:
for i, quant_feature in tqdm(enumerate(quantile_feats)):
    append_static_pred_relative_to_df(train, train, quant_feature, f"static_pred_{i}")

In [ ]:
static_pred_feats = [f"static_pred_{i}" for i in range(len(quantile_feats))]

In [ ]:
if not FULL_TRAIN:
    for i, quant_feature in tqdm(enumerate(quantile_feats)):
        append_static_pred_relative_to_df(valid, train, quant_feature, f"static_pred_{i}")

### Append speed features (if USE_SPEED_FEATURE = True)

In [ ]:
if USE_SPEED_FEATURES:
    supersegment_speed_features = pd.read_parquet(data_dir / "traffic" / city_name / "ss_speeds.parquet")

In [ ]:
features = [
    # Edge position features
    "counter_distance_euclidean",
    "counter_distance_euclidean_mean_all",
    "x",
    "y",
    "supersegment_id",
    # Traffic features
    # "quantile",
    "total_traffic",
    "h3_vol",
    # "city_volumes_gr",
    # "city_volumes_sum",
    # "h3"
] + [f for f in train.columns if f.startswith("PC")] + quantile_feats[-1:] + static_pred_feats

if USE_SPEED_FEATURES:
    speed_feats = ["dummy_eta", "dummy_eta_freeflow", "segment_count", "length", "lanes"]
    features = features + speed_feats

label = "eta"

In [ ]:
if USE_SPEED_FEATURES:
    train = train.merge(supersegment_speed_features, left_on="supersegment_id", right_index=True)
    if not FULL_TRAIN:
        valid = valid.merge(supersegment_speed_features, left_on="supersegment_id", right_index=True)

<h1> Prepare LGB Datasets and train the model </h1>

In [ ]:
if FULL_TRAIN:
    lgb_set = lgb.Dataset(train[features], train[label], init_score=train[static_pred_feats[0]])
else:
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(train[features], train[label], init_score=train[static_pred_feats[0]])
    lgb_eval = lgb.Dataset(valid[features], valid[label], reference=lgb_train, init_score=valid[static_pred_feats[0]])

In [ ]:
callbacks = []

SAVE_MODEL_CHECKPOINTS = False

if SAVE_MODEL_CHECKPOINTS:
    def save_model_callback(env):
        if env.iteration % 100 == 0:
            env.model.save_model(data_dir / "models" / model_name / city_name / f"modelQ_{env.iteration}.lgb")

    callbacks.append(save_model_callback)

In [ ]:
import time

PRINT_TIME = True

if PRINT_TIME:
    def print_time(env):
        print(time.time() - START_TIME)
    
    callbacks.append(print_time)

In [ ]:
# When training on the full data, will save evaluation time by only validating the model on a small data sample
#  when USE_MOCK_VALIDATION_WHEN_FULL_TRAIN is True. Otherwise, will validate each time on the whole training dataset
USE_MOCK_VALIDATION_WHEN_FULL_TRAIN = True
SMALL_SAMPLE_SIZE = 20

if USE_MOCK_VALIDATION_WHEN_FULL_TRAIN:
    lgb_set_mock = lgb.Dataset(
        data=train[:SMALL_SAMPLE_SIZE][features], 
        label=train[:SMALL_SAMPLE_SIZE][label], 
        init_score=train[:SMALL_SAMPLE_SIZE][static_pred_feats[0]]
    )
    lgb_val = lgb_set_mock
else:
    lgb_val = lgb_set

In [ ]:
# From Optuna
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    'metric': 'l1',
    'num_leaves': num_leaves[city_name],
    'learning_rate': 0.1,
    'feature_fraction': 1.0,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'lambda_l1': 8.544245989665272,
    'lambda_l2': 0.09577740930772316,
    'min_child_samples': 10,
}

print('Starting training...')

START_TIME = time.time()

if FULL_TRAIN:
    gbm = lgb.train(params,
                    lgb_set,
                    num_boost_round=num_iters[city_name],
                    valid_sets=[lgb_val],
                    callbacks=callbacks,
                    verbose_eval=25)
else:
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=NO_LGB_ITERS,
                    valid_sets=[lgb_eval],
                    callbacks=[lgb.early_stopping(stopping_rounds=200)],
                    verbose_eval=25)

### Explain model predictions using SHAP

In [ ]:
import shap
def shap_wrapped(data, model, features):
    explainer = shap.TreeExplainer(model)

    X = data.sample(500)print_timees]

    shap_values = explainer.shap_values(X)
    
    shap.initjs()    
    shap.summary_plot(shap_values, X)
    shap.summary_plot(shap_values, X, plot_type="bar")
    
shap_wrapped(train, gbm, features)

### Save model artifact

In [ ]:
model_path = data_dir / "models" / model_name / city_name / "modelQ.lgb"
model_path.parent.mkdir(parents=True, exist_ok=True)
gbm.save_model(model_path)
print(f"Saved {model_name} at {model_path}")

# Create submission

In [ ]:
# When using a saved artifact to do predictions:
# gbm = lgb.Booster(model_file=data_dir / "models" / model_name / city_name / "model.lgb")

In [ ]:
test_left = pd.DataFrame({"identifier": list(supersegment_to_id.keys())})
test_left = pd.concat([test_left]*100)
test_idx = []
current = 0
unique_segments = test_left["identifier"].nunique()
print(unique_segments)
for i in range(100):
    test_idx.extend([current]*unique_segments)
    current += 1
test_left["test_idx"] = test_idx

assert test_idx[unique_segments-1] != test_idx[unique_segments]
test_left["supersegment_id"] = [supersegment_to_id[s] for s in test_left["identifier"]]

# Get supersegment features like nearest_counter_id
test_left = test_left.merge(supersegments, on="supersegment_id")
test_left

In [ ]:
counts_test, _ = load_data(city_name, "test")
# Get counter_id of nodes
counts_test = counts_test.merge(nodes_with_counters[["node_id", "counter_id"]], on="node_id")
counts_test

In [ ]:
# Join label stub to counters
test = test_left.merge(counts_test, left_on=["test_idx", "nearest_counter_id"], right_on=["test_idx", "counter_id"], how="left")
print(test.shape)

In [ ]:
assert unique_segments * 100 == len(test)

In [ ]:
test, _ = create_multi_traffic_state(test, traffic_quantiles_list, "volumes_mdn", "test")
print(test.count())

In [ ]:
test = create_h3_variable(test, mode="test")
test.count()

In [ ]:
# Some counters have missing data, let's fill this in a dummy way
features_to_fill = ["node_id", "counter_id", "volumes_gr", "volumes_mdn", "volumes_sum", "volumes_last"]
test[features_to_fill] = test.groupby("supersegment_id")[features_to_fill].transform(lambda x: x.ffill().bfill())
assert test.count().sum() == len(test.columns) * len(test)
test.count()
# TODO doesn't match for Madrid!
# Some (required) nodes are missing counters for whole test set. Should we use blacklisting?
# test[features_to_fill] = test.groupby("supersegment_id")[features_to_fill].transform(lambda x: x.ffill().bfill())

In [ ]:
test = merge_pcas(city_name, test, mode="test")

In [ ]:
# # Add city avg volume feats just in case
# test["city_volumes_gr"] = test.groupby("test_idx")["volumes_gr"].transform(np.median)
# test["city_volumes_sum"] = test.groupby("test_idx")["volumes_sum"].transform(np.median)

In [ ]:
for i, quant_feature in enumerate(quantile_feats):
    append_static_pred_relative_to_df(test, train, quant_feature, f"static_pred_{i}")

In [ ]:
if USE_SPEED_FEATURES:
    test = test.merge(supersegment_speed_features, left_on="supersegment_id", right_index=True)

In [ ]:
for f in features:
    assert f in test.columns

In [ ]:
gbm_preds = gbm.predict(test[features])
test["eta"] = (gbm_preds + test["static_pred_0"]).round(2)

In [ ]:
test[["identifier", "eta", "test_idx"]].head()

In [ ]:
submission_folder = data_dir / 'submissions' / model_name / city_name / 'labels'
submission_folder.mkdir(exist_ok=True, parents=True)
test[["identifier", "eta", "test_idx"]].to_parquet(submission_folder / f'eta_labels_test_Q_{ITER}.parquet', compression='snappy')